## Load Data

In [1]:
import os
import json
import pandas as pd
from glob import glob
import numpy

# Define the base directories
base_dirs = ['train', 'eval', 'test']

for i in range(len(base_dirs)):
    base_dirs[i] = 'data/methods2test_data/' + base_dirs[i]
# List to collect all data (rows)
data = []

total_exmaples = 0

is_broken = False
base_dir = 'data/methods2test_data/train'
# Iterate over all subdirectories (examples) in the base directory
for example_dir in os.listdir(base_dir):
    example_path = os.path.join(base_dir, example_dir)
    
    # Ensure we're dealing with directories (i.e., each example is its own directory)
    if os.path.isdir(example_path):
        # Use glob to get all JSON files in the example directory
        json_files = glob(os.path.join(example_path, '*.json'))
        
        # Iterate over the JSON files
        for json_file in json_files:
            # Read the JSON data
            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            # Add the JSON data to the data list (can modify if additional info is needed)
            data.append(json_data)
            total_exmaples+=1
            if(total_exmaples>=15):
                print("breaking")
                is_broken = True
                break
    if(is_broken):
        break
# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(data)

# Optional: Inspect the first few rows
print(df.head())

breaking
                                          test_class  \
0  {'identifier': 'SourceCodeWriterTest', 'superc...   
1  {'identifier': 'RepositoryModelTest', 'supercl...   
2  {'identifier': 'RepositoryModelTest', 'supercl...   
3  {'identifier': 'ReflectionUtilsTest', 'supercl...   
4  {'identifier': 'JMSMessagePropertySupportTest'...   

                                           test_case  \
0  {'identifier': 'testWrite', 'parameters': '()'...   
1  {'identifier': 'testRemoveAll', 'parameters': ...   
2  {'identifier': 'testRemoveAll', 'parameters': ...   
3  {'identifier': 'testHasSuperClass', 'parameter...   
4  {'identifier': 'testCheckValidObjectList', 'pa...   

                                         focal_class  \
0  {'identifier': 'SourceCodeWriter', 'superclass...   
1  {'identifier': 'RepositoryModel', 'superclass'...   
2  {'identifier': 'RepositoryModel', 'superclass'...   
3  {'identifier': 'ReflectionUtils', 'superclass'...   
4  {'identifier': 'JMSMessageProperty

In [2]:
focal_methods = df['focal_method']
code_bodies = pd.DataFrame({'code': focal_methods.apply(lambda x: x['body'])})

In [3]:
test_cases = df["test_case"]
test_code_bodies = pd.DataFrame({'tests': test_cases.apply(lambda x: x['body'])})

In [ ]:
code_test_df = pd.concat([code_bodies, test_code_bodies], axis=1)
code_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    15 non-null     object
 1   tests   15 non-null     object
dtypes: object(2)
memory usage: 368.0+ bytes


In [5]:
code_test_df['prompted_code'] = "Here is a method implementation in Java:\n\n" + code_test_df['code'] + "\n\nWrite a full test class with test cases to validate the method defined above."

### Codellama Prompting through together.ai

In [14]:
import openai
import os
import json
from dotenv import load_dotenv
load_dotenv()

TOGETHER_API_KEY = "6fb53d29d754856e6738e69e825c22ee935d69652f544f9a3d049454a0ee8bc1"
 
client = openai.OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url="https://api.together.xyz/v1",
)

In [15]:
def get_code_completion(messages, max_tokens=512, model="codellama/CodeLlama-7b-Instruct-hf"):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        stop=[
            "<step>"
        ],
        frequency_penalty=1,
        presence_penalty=1,
        top_p=0.7,
        n=10,
        temperature=0.7,
    )
 
    return chat_completion

In [16]:
messages = [
      {
            "role": "system",
            "content": "You are an expert programmer that helps to generate Java unit tests based on the user's code, with concise explanations. Don't be too verbose.",
      },
      {
            "role": "user",
            "content": "Here is a method implementation in Java:\n\n" + code_test_df['code'] + "\n\nWrite a full test class with test cases to validate the method defined above.",
      }
]
 
chat_completion = get_code_completion(messages)
            
print(chat_completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'id': '902771459d3c22d0', 'error': {'message': 'Unable to access non-serverless model codellama/CodeLlama-7b-Instruct-hf. Please visit https://api.together.ai/models/codellama/CodeLlama-7b-Instruct-hf to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}